In [1]:
# import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import pickle
import gzip

In [2]:
with open('../input/ctr-read-data/merged_df.pkl', 'rb') as inputfile:
    merged_df = pickle.load(inputfile)

In [3]:
train_df = merged_df.loc[:40000000, :]
test_df = merged_df.loc[40000001:, :]

In [4]:
def calc_smooth_mean(df, col, label, mean_label, weight):
    # Compute the number of values and the mean of each group
    agg = df.groupby(col)[label].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + weight * mean_label) / (counts + weight)

    mapping_dict = {key:value for key,value in zip(smooth.index.tolist(),smooth.values.tolist())}
    # Replace each value by the according smoothed mean and return mapping dict
    return df[col].map(smooth), mapping_dict

In [5]:
# Compute the global mean
mean_label = train_df["label"].mean()
list_mapping_dict = []

for col in train_df.columns.tolist():
    encoded, mapping_dict = calc_smooth_mean(train_df, col, "label", mean_label, 300)
    downcasted = pd.to_numeric(encoded , downcast='float')
    train_df[col+'_tenc'] = downcasted
    list_mapping_dict.append(mapping_dict)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
train_df3 = train_df.loc[10000001:15000000,:]

In [7]:
train_df3.to_csv("train_df3.csv")